# CONEXÃO

In [ ]:
import psycopg2
import fdb
from dotenv import load_dotenv
import os

credentials = load_dotenv()

# Conexões
cur_dest = fdb.connect(
    user=os.getenv("FDB_USER"),
    password=os.getenv("FDB_PASS"),
    host=os.getenv("FDB_HOST"),
    port=os.getenv("FDB_PORT"),
    database=os.getenv("FDB_PATH"),
    charset="WIN1252"
).cursor()

cur_orig = psycopg2.connect(
    user=os.getenv("PG_USER"),
    password=os.getenv("PG_PASS"),
    host=os.getenv("PG_HOST"),
    database=os.getenv("PG_DB"),
    schema=os.getenv("PG_SCHEMA")
).cursor()

def commit():
    cur_dest.commit()

# FERRAMENTAS

In [ ]:
global cadest, empresa
cadest = {}
empresa = cur_dest.execute("SELECT empresa FROM cadcli").fetchone()[0]

def limpa_tabela(tabelas):
    for tabela in tabelas:
        cur_dest.execute(f"DELETE FROM {tabela}")
    commit()

def cria_coluna(tabela, coluna):
    try:
        cur_dest.execute(f"ALTER TABLE {tabela} ADD {coluna} VARCHAR(255)")
    except fdb.DatabaseError as e:
        print(f"Erro ao criar coluna {coluna} na tabela {tabela}: {e}")
    else:
        commit()

# COMPRAS
<p>Extração, tratamento e carregamento dos dados referentes ao módulo compras</p>

## CADASTROS BASE

### CADUNIMEDIDA

In [ ]:
limpa_tabela(("cadunimedida"))

insert = cur_dest.prepare("insert into cadunimedida(sigla,descricao) values(?,?)")

cur_orig.execute("select distinct replace(unidade, ' ', 'NULL'), nome_unidemb from cgamercadorias order by 1")

for row in cur_orig:
    cur_dest.execute(insert, (row[0], row[1]))
commit()


### GRUPO E SUBGRUPO

In [ ]:
limpa_tabela(("cadgrupo", "cadsubgrupo"))

cur_orig.execute("""
select
	distinct
	to_char(grupo::integer, '000') grupo,
	subgrupo,
	a.descricao
from
	cgasubgrupos a
union all
select distinct
	to_char(grupo::integer, '000') grupo,
	concat(substring(mercadoria,1,1), substring(subgrupo,2,2)) subgrupo,
	b.descricao
	--, a.mercadoria
from
	cgc_vw_mercadorias a
left join cgasubgrupos b using (grupo, subgrupo)
where a.mercadoria not like '0%'                  
union all
	select
	distinct
	to_char(grupo::integer, '000') grupo,
	'000',
	a.descricao
from
	cgagrupos a
order by
	2,
	1
""")

for row in cur_orig:
    if row.subgrupo == '000':
        cur_dest.execute("insert into cadgrupo(grupo, nome) values(?, ?)", (row.grupo, row.descricao))
        commit()
    else:
        cur_dest.execute("insert into cadsubgrupo(grupo, subgrupo, nome) values(?, ?, ?)", (row.grupo, row.subgrupo, row.descricao))
commit()

### CADEST

In [ ]:
limpa_tabela(("cadest"))

insert = cur_dest.prepare("insert into cadest(grupo, subgrupo, codigo, cadpro, codreduz, disc1, ocultar, unid1, tipopro, usopro) values (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)")

cur_orig.execute("""
select
	to_char(grupo::integer, '000') grupo,
	concat(substring(mercadoria,1,1), substring(subgrupo,2,2)) subgrupo,
	substring(mercadoria, 2, 3) codigo,
	concat(
		to_char(grupo::integer, '000'),'.',concat(substring(mercadoria,1,1), substring(subgrupo,2,2)),'.',substring(mercadoria, 2, 3)
	) cadpro,
	idespecificacao codreduz,
	descricao disc1,
	case when inativo = 0 then 'N' else 'S' end ocultar,
	unidade unid1
from
	cgc_vw_mercadorias
order by 1, 2, 3
""")

for row in cur_orig:
    cur_dest.execute(insert, (row.grupo, row.subgrupo, row.codigo, row.cadpro, row.codreduz, row.disc1, row.ocultar, row.unid1, 'P', 'C'))
    cadest[row.codreduz] = row.cadpro
commit()

### CENTRO DE CUSTO

In [ ]:
limpa_tabela(("centrocusto"))

try:
    cur_dest.execute(f"insert into destino(cod,desti,empresa) values('000000000','CONVERSAO',{empresa})")
except fdb.DatabaseError as e:
    print(f"Erro ao inserir destino: {e}")
else:
    commit()

insert = cur_dest.prepare("""
    insert into centrocusto (codccusto, descr, ccusto, ocultar, empresa, poder, orgao, destino, pai) values (?, ?, ?, ?, ?, ?, ?, ?, ?)
""")

cur_orig.execute("""
with CTE as (
    select
        id as codccusto,
        nome as descr,
        pai,
        versao,
        row_number() over (partition by id order by versao desc) as rn
    from
        pinhalpm.cadlocalfisico
)
select
    codccusto,
    descr,
    pai
from
    CTE a
where
    rn = 1;
""")

for row in cur_orig:
    cur_dest.execute(insert, (row.codccusto, row.descr, '001', 'N', empresa, '02', '01', '000000000', row.pai))
commit()

### ALMOXARIFADOS

In [ ]:
limpa_tabela(("destino"))
cria_coluna("destino", "almox_pai")

insert = cur_dest.prepare("""insert into destino(cod,desti,empresa,almox_pai) values(?,?,?,?)""")

cur_orig.execute("""
select
	to_char(id, 'fm000000000') cod,
	nome desti,
	g.unidadeprincipal pai,
    localfisico codccusto
from
	gapunidadealmox g  
""")

for row in cur_orig:
    cur_dest.execute(insert, (row.cod, row.desti, empresa, row.pai))
    cur_dest.execute("insert into centrocusto_destino (codccusto, destino) values (?, ?)", (row.codccusto, row.cod))
commit()

## COTAÇÃO